In [ ]:
import pandas as pd
import time
import pickle
import logging
import sys
from datetime import datetime
from datetime import timedelta


In [ ]:
# bike trips
filename_DD = f"../data/nextbike/hourly_demand_supply_Dresden 2025-03-19_10-47-56.csv"
filename_FB = f"../data/nextbike/hourly_demand_supply_Freiburg_missing_interpolated_2025-03-19_10-47-56.csv"
df_DD = pd.read_csv(filename_DD, index_col=None, parse_dates=["datetime_hour"])
df_FB = pd.read_csv(filename_FB, index_col=None, parse_dates=["datetime_hour"])
df_DD = df_DD.sort_values("datetime_hour")
df_FB = df_FB.sort_values("datetime_hour")
# events 
filename_events = "../data/events/df_events_with_hex_id_and_all_cols_2025-04-23_13-12-48.csv"
df_events = pd.read_csv(filename_events, index_col=None, parse_dates=["StartDateTime", "EndDateTime"])



In [ ]:
initial_len_df_DD = len(df_DD)
initial_len_df_FB = len(df_FB)

In [ ]:
# weather
filename_weather_DD = "../data/weather/df_Dresden_weather_hourly 2025-03-28_20-51-37.csv"
filename_weather_FB = "../data/weather/df_Freiburg_weather_hourly 2025-03-28_20-51-37.csv"

df_weather_DD = pd.read_csv(filename_weather_DD, index_col=None, parse_dates=["datetime_hour"])
df_weather_FB = pd.read_csv(filename_weather_FB, index_col=None, parse_dates=["datetime_hour"])

df_weather_DD = df_weather_DD.drop(columns=["Precipitation", "Wind"])
df_weather_FB = df_weather_FB.drop(columns=["Precipitation", "Wind"])


df_DD = df_DD.merge(df_weather_DD, on="datetime_hour", how="left")
df_FB = df_FB.merge(df_weather_FB, on="datetime_hour", how="left")

In [ ]:
## calendar effects
for i, df_tmp in enumerate([df_DD, df_FB]):
    df_tmp["weekday"] = df_tmp.datetime_hour.dt.dayofweek
    df_tmp["weekday"] = df_tmp["weekday"].map({0: "Mon", 1: "Tue", 2: "Wed", 3: "Thu", 4: "Fri", 5: "Sat", 6: "Sun"})
    weekday_df = pd.get_dummies(df_tmp["weekday"], prefix="weekday", drop_first=False, dtype=int)
    weekday_df.index = df_tmp.index
    weekday_df.drop(columns="weekday_Mon", inplace=True)
    df_tmp[weekday_df.columns] = weekday_df

    df_tmp["hour"] = df_tmp.datetime_hour.dt.hour
    hours_df = pd.get_dummies(df_tmp["hour"], prefix="hour", drop_first=False, dtype=int)
    hours_df.index = df_tmp.index
    hours_df.drop(columns="hour_0", inplace=True)
    df_tmp[hours_df.columns] = hours_df
    df_tmp["is_dayoff"] = df_tmp["weekday_Sat"] + df_tmp["weekday_Sun"]
    # list of german holidays in 2023 and 2024
    if i == 0:
        # holidays for Dresden
        german_holidays = ["2024-01-01", "2024-03-29", "2024-04-01", "2024-05-01", "2024-05-09", "2024-05-20", "2024-10-03", "2024-10-31"]
    else:
        german_holidays = ["2023-06-08", "2024-10-03"]
    german_holidays = [pd.to_datetime(date).date() for date in german_holidays]
    flt = df_tmp.datetime_hour.dt.date.isin(german_holidays)
    len(df_tmp[flt])
    df_tmp.loc[flt, "is_dayoff"] = 1

# modify events

In [ ]:
df_events

In [ ]:
flt = df_events.EndDateTime < df_events.StartDateTime
#  replace date of EndDateTime with next day but keep time

df_events.loc[flt, "EndDateTime"] = df_events.loc[flt, "EndDateTime"] + pd.DateOffset(days=1)

In [ ]:
flt = df_events.EndDateTime < df_events.StartDateTime
assert len(df_events[flt])==0

In [ ]:
flt = df_events.EndDateTime.isna()
df_events.loc[flt, "EndDateTime"] = df_events.loc[flt, "StartDateTime"] + pd.DateOffset(hours=3)

In [ ]:
df_events.StartDateTime = df_events.StartDateTime.dt.floor("h")
df_events.EndDateTime = df_events.EndDateTime.dt.floor("h")

In [ ]:
df_events.EventCategory.fillna("Unknown", inplace=True)

In [ ]:
df_events["category"] = df_events.SourceGroup + " - " + df_events.EventCategory

In [ ]:
df_events["category"].value_counts(dropna=False)

In [ ]:
flt = df_events.category == "deecooob GmbH - Unknown"
df_events.loc[flt, "category"] = "deecoob GmbH - " + df_events.loc[flt, "Source"]


In [ ]:
df_events["major_category"] =df_events.category

In [ ]:
df_events.major_category = df_events.major_category.replace(to_replace=["deecoob GmbH - eventim.de",
                                            "deecoob GmbH - bandsintown.com",
                                            "deecoob GmbH - jambase.com",
                                             "deecoob GmbH - setlist.fm",
                                            "deecoob GmbH - concertful.com", "deecoob GmbH - eventbrite.com",
                                            "deecoob GmbH - songkick.com",	"deecoob GmbH - ticketmaster.de", "deecoob GmbH - wegow.com"],
                                value="deecoob GmbH - Music events")

df_events.major_category = df_events.major_category.replace(to_replace="deecoob GmbH - facebook.com",
                                value="deecoob GmbH - Facebook events")



df_events.major_category = df_events.major_category.replace(["Self scraped - Feste, Märkte & Messen", 
                                                             "Self scraped - Feste"
                                                             ], "Self scraped - Festivals, fairs and markets")

df_events.major_category = df_events.major_category.replace("Self scraped - Ausstellungen & Museen", "Self scraped - Exhibitions & Museums")

df_events.major_category = df_events.major_category.replace("GEMA - Freizeit/Sport/Kultur", "GEMA - Leisure, sport and culture")
df_events.major_category = df_events.major_category.replace("GEMA - Handel und Dienstleistungen", "GEMA - Trade and services")

df_events.major_category = df_events.major_category.replace(to_replace=["Self scraped - Unknown", 
                                                                      "Self scraped - Verschiedenes",
                                                                       "Self scraped - Konzerte","Self scraped - Vorträge & Lesungen",
                                                                       "Self scraped - Musik & Party",
                                                                      "Self scraped - Kultur", "Self scraped - Theater & Schauspiel",
                                                                       "Self scraped - Verschiedenes ", "Self scraped - Kabarett & Comedy", 
                                                                       "Self scraped - Burgen & Schlösser"],
                                value="Self scraped - Other")


df_events.major_category = df_events.major_category.replace(to_replace=["GEMA - Kirchen",
                                                                      "GEMA - Gesundheitswesen (Ärzte)",
                                                                      "GEMA - Gesundheitswesen (Krankenhäuser)" ,
                                                                      "GEMA - Unknown"  ,
                                                                      "GEMA - Sozialeinrichtungen"   ,
                                                                      "GEMA - Staat und Verwaltung" ,
                                                                      "GEMA - Nicht geographisch",
                                                                      "GEMA - Bildungseinrichtungen", "GEMA - Mobiler Nutzungsort"
                                                                      ],
                                value="GEMA - Other")



df_events.major_category = df_events.major_category.replace(to_replace=[    "GEMA - Gastronomie (mit Übernachtung)",
                                                                        "GEMA - Gastronomie (ohne Übernachtung)",
                                                                      ],
                                value="GEMA - Restaurants & Hotels")


df_events.major_category = df_events.major_category.replace(to_replace=[    "Self scraped - Football match",
                                                                        "Self scraped - Sportveranstaltungen",
                                                                      ],
                                value="Self scraped - Sports events")



In [ ]:
df_events.major_category.value_counts(dropna=False).sort_index()

In [ ]:
excluded_categories = ["Self scraped - Exhibitions & Museums", "Self scraped - Other"] # maybe "GEMA - Trade and services"
df_events = df_events.loc[~df_events.major_category.isin(excluded_categories)].copy()

In [ ]:
df_events.drop(columns = ["_merge", "lat", "lon", "PLZ_GEMA", 
                         "Datum_Nutzungsbeginn_GEMA", "Datum_Nutzungsende_GEMA", "Ort_GEMA", "Strasse_GEMA",
                         "Hausnummer_GEMA", "EndDate", "EndTime", "HouseNumber", "Street_orig", "StreetHouseNumber_orig", "Street", "StreetHouseNumber"], errors="ignore", inplace=True)

In [ ]:
df_events

In [ ]:
df_events["StartDateTime_adj_1"] = df_events.StartDateTime - pd.Timedelta(hours=1)

In [ ]:
df_events["StartDateTime_adj_2"] = df_events.StartDateTime - pd.Timedelta(hours=2)

In [ ]:
df_events_grouped_start_1 = df_events.groupby(["hex_id", "StartDateTime_adj_1"]).size().reset_index(name="event_count_start_1")
df_events_grouped_start_2 = df_events.groupby(["hex_id", "StartDateTime_adj_2"]).size().reset_index(name="event_count_start_2")
df_events_grouped_end = df_events.groupby(["hex_id", "EndDateTime"]).size().reset_index(name="event_count_end")

In [ ]:
def transform_join(df_input):
    df_tmp =  df_input.copy()
    df_tmp = df_tmp.merge(df_events_grouped_start_1, how="left", left_on=["datetime_hour", "hex_id"], right_on=["StartDateTime_adj_1", "hex_id"], indicator=False)
    df_tmp.event_count_start_1.fillna(0, inplace=True)

    df_tmp = df_tmp.merge(df_events_grouped_start_2, how="left", left_on=["datetime_hour", "hex_id"], right_on=["StartDateTime_adj_2", "hex_id"], indicator=False)
    df_tmp.event_count_start_2.fillna(0, inplace=True)

    df_tmp = df_tmp.merge(df_events_grouped_end, how="left", left_on=["datetime_hour", "hex_id"], right_on=["EndDateTime", "hex_id"], indicator=False)
    df_tmp.drop(columns=["StartDateTime_adj_1", "StartDateTime_adj_2", "EndDateTime"], inplace=True, errors="ignore")
    df_tmp.event_count_end.fillna(0, inplace=True)
    return df_tmp


In [ ]:
df_DD = transform_join(df_DD)
df_FB = transform_join(df_FB)


In [ ]:
def transform(df_input):
    df_tmp = df_input.copy()
    df_tmp["event_count_start"] = df_tmp.event_count_start_1+df_tmp.event_count_start_2
    df_tmp.drop(columns=["event_count_start_1", "event_count_start_2"], inplace=True)
    weekday_cols = [col for col in df_tmp.columns if col.startswith("weekday")]
    df_tmp.drop(columns=weekday_cols, inplace=True)
    return df_tmp

In [ ]:
df_DD 

In [ ]:
df_DD.describe()

In [ ]:
df_DD = transform(df_DD)
df_FB = transform(df_FB)


In [ ]:
assert len(df_DD) == initial_len_df_DD
assert len(df_FB) == initial_len_df_FB

In [ ]:
df_DD

In [ ]:
df_DD = df_DD.sort_values("datetime_hour")
df_FB = df_FB.sort_values("datetime_hour")

In [ ]:
time = datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
df_DD.to_csv(f"../data/df_DD_for_SARIMAX_with_adj_events_{time}.csv", index=False)
df_FB.to_csv(f"../data/df_FB_for_SARIMAX_with_adj_events_{time}.csv", index=False)